- Sequence-to-Sequence  
    - seq2seq는 번역기에서 대표적으로 사용되는 모델
    - RNN을 어떻게 조립했느냐에 따라서 seq2seq라는 구조가 만들어짐

- seq2seq는 크게 인코더와 디코더라는 두 개의 모듈로 구성됨
    - 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 함
    - 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송함
    - 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력함
    - 인코더 아키텍처와 디코더 아키텍처의 내부는 사실 두 개의 RNN 아키텍처
    - 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 출력하는 RNN 셀을 디코더라고 함
    - 성능 문제로 인해 실제로는 바닐라 RNN이 아니라 LSTM 셀 또는 GRU 셀들로 구성됨
  
![a](https://wikidocs.net/images/page/24996/%EB%8B%A8%EC%96%B4%ED%86%A0%ED%81%B0%EB%93%A4%EC%9D%B4.PNG)
  
- 인코더는 입력 문장은 단어 토큰화를 통해서 단어 단위로 쪼개지고 단어 토큰 각각은 RNN 셀의 각 시점의 입력이 됨
    - 인코더 RNN 셀은 모든 단어를 입력받은 뒤에 인코더 RNN 셀의 마지막 시점의 은닉 상태를 디코더 RNN 셀로 넘겨주는데 이를 컨텍스트 벡터라고 함
    - 컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태에 사용됨
  
- 디코더는 기본적으로 RNNLM(RNN Language Model)
    - 디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 \<sos\>가 들어감
    - 디코더는 \<sos\>가 입력되면, 다음에 등장할 확률이 높은 단어를 예측함
    - 첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였음
    - 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력함
    - 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냄
    - 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복함
    - 이 행위는 문장의 끝을 의미하는 심볼인 \<eos\>가 다음 단어로 예측될 때까지 반복됨

- seq2seq는 훈련 과정과 테스트 과정(또는 실제 번역기를 사람이 쓸 때)의 작동 방식이 조금 다름
    - 훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답인 상황인 \<sos\> je suis étudiant를 입력 받았을 때, je suis étudiant \<eos\>가 나와야 된다고 정답을 알려주면서 훈련함
    - 이를 교사 강요(teacher forcing)라고 함
    - 반면, 테스트 과정에서는 앞서 설명한 과정과 같이 디코더는 오직 컨텍스트 벡터와 \<sos\>만을 입력으로 받은 후에 다음에 올 단어를 예측하고, 그 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복함

![rnn근황.PNG](https://wikidocs.net/images/page/24996/rnn%EA%B7%BC%ED%99%A9.PNG)  
  
- 현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듦
- 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있음
- 그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냄
- 이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있음
    - 그렇기 때문에 앞서 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있음
- 디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용함
- 디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 \<sos\>로부터, 다음에 등장할 단어를 예측함
    - 이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값이 되고, 이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터. 즉, 또 다시 다음에 등장할 단어를 예측하게 될 것

![b](https://wikidocs.net/images/page/24996/decodernextwordprediction.PNG)

- 출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있음. seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 함
- 디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정함

문자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기

병렬 코퍼스 데이터에 대한 이해와 전처리  
  
- 병렬 데이터라고 하면 앞서 수행한 태깅 작업의 데이터를 생각할 수 있지만, 앞서 수행한 태깅 작업의 병렬 데이터와 seq2seq가 사용하는 병렬 데이터는 성격이 조금 다름
- 태깅 작업의 병렬 데이터는 쌍이 되는 모든 데이터가 길이가 같았지만 여기서는 쌍이 된다고 해서 길이가 같지않음
- 실제 번역기를 생각해보면 구글 번역기에 '나는 학생이다.'라는 토큰의 개수가 2인 문장을 넣었을 때 'I am a student.'라는 토큰의 개수가 4인 문장이 나오는 것과 같음
- seq2seq는 기본적으로 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있다고 가정함

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

2023-03-08 12:12:07.586826: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 12:12:07.751727: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 12:12:07.791869: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-08 12:12:08.546272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('sample count : ', len(lines))

sample count :  208906


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]
lines.sample(5)

,src,tar
16406,They're special.,Elles sont spéciales.
31237,I signed the check.,J'ai signé le chèque.
43569,How did you get here?,Comment êtes-vous parvenue ici ?
8357,Keep it short.,Soyez bref.
52997,I've had a lot of fun.,Je me suis beaucoup amusé.


In [5]:
# 시작을 의미하는 심볼 <sos>과 종료를 의미하는 심볼 <eos>을 넣어주어야 함.
# 여기서는 <sos>와 <eos> 대신 \t를 시작 심볼, \n을 종료 심볼로 간주하여 추가하고 
# 다시 데이터를 출력.

lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(5)

,src,tar
52683,I'm dreading the exam.,\t J'appréhende l'examen. \n
3620,It may rain.,\t Il pourrait pleuvoir. \n
29413,Can I use your pen?,\t Puis-je utiliser ton stylo ? \n
45955,Let me rest a little.,\t Laissez-moi me reposer un peu. \n
31025,I like to be early.,\t J'aime être en avance. \n


In [6]:
# 문자 집합 생성.
# 단어 집합이 아니라 문자 집합이라고 하는 이유는 토큰 단위가 단어가 아니라 문자이기 때문

src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
        
# 문자 집합의 크기
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print('source 문장의 char 집합 : ', src_vocab_size)
print('target 문장의 char 집합 : ', tar_vocab_size)

source 문장의 char 집합 :  80
target 문장의 char 집합 :  104


In [7]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [8]:
# 인덱스 부여

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [9]:
# 정수 인코딩

encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    for char in line:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩:', encoder_input[:5])

source 문장의 정수 인코딩: [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [10]:
# 디코더의 입력이 될 프랑스어 데이터에 대해 정수 인코딩

decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩:', decoder_input[:5])

target 문장의 정수 인코딩: [[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [11]:
# 디코더의 예측값과 비교하기 위한 실제값 필요. 실제값에는 시작 심볼에 해당되는 <sos>가 있을 필요가 없음
# 그래서 정수 인코딩 과정에서 <sos>를 제거함. 모든 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거

decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 : ', decoder_target[:5])

target 문장 레이블의 정수 인코딩 :  [[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2]]


In [12]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 23
target 문장의 최대 길이 : 76


In [13]:
# 패딩할 때 이 두 개의 데이터의 길이를 전부 동일하게 맞춰줄 필요는 없음
# 영어는 영어샘플끼리, 프랑스어는 프랑스어 샘플끼리 길이를 맞춰 패딩

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [14]:
# 원-핫 인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

교사 강요(Teacher forcing)  

- 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용
    - 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 함
    - 이런 상황이 반복되면 훈련 시간이 느려짐
    - 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있음
- 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 함

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [16]:
encoder_inputs = Input(shape=(None, src_vocab_size))
# LSTM의 은닉 상태 크기 256.
# 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True 설정
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs 는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 상태가 은닉 상태와 셀 상태로 나뉨.
# 두 가지 상태를 encoder_states에 저장. 이 두 가지 상태 모두를 디코더로 전달. 
# 이것이 컨텍스트 벡터임.
encoder_states = [state_h, state_c]

2023-03-08 12:12:17.843372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 12:12:19.066031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9637 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:5e:00.0, compute capability: 7.5
2023-03-08 12:12:19.066740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9636 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5


In [17]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# initial_state의 인자값으로 encoder_states를 주는 코드가 디코더에게 
# 인코더의 은닉 상태, 셀 상태 전달하여 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용함
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 출력층에 프랑스어의 단어 집합의 크기만큼 뉴런을 배치한 후
# 소프트맥스 함수를 사용하여 실제값과의 오차를 구함
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [18]:
# 입력으로는 인코더 입력과 디코더 입력이 들어가고, 디코더의 실제값인 decoder_target도 필요함.
model.fit(x=[encoder_input, decoder_input], y=decoder_target,
          batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40


2023-03-08 12:12:27.557582: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


750/750 [==============================] - 24s 27ms/step - loss: 0.7348 - val_loss: 0.6457
Epoch 2/40
750/750 [==============================] - 19s 25ms/step - loss: 0.4524 - val_loss: 0.5248
Epoch 3/40
750/750 [==============================] - 19s 25ms/step - loss: 0.3769 - val_loss: 0.4599
Epoch 4/40
750/750 [==============================] - 19s 25ms/step - loss: 0.3336 - val_loss: 0.4211
Epoch 5/40
750/750 [==============================] - 19s 25ms/step - loss: 0.3052 - val_loss: 0.3983
Epoch 6/40
750/750 [==============================] - 18s 24ms/step - loss: 0.2846 - val_loss: 0.3815
Epoch 7/40
750/750 [==============================] - 19s 25ms/step - loss: 0.2684 - val_loss: 0.3694
Epoch 8/40
750/750 [==============================] - 18s 24ms/step - loss: 0.2555 - val_loss: 0.3603
Epoch 9/40
750/750 [==============================] - 18s 24ms/step - loss: 0.2445 - val_loss: 0.3539
Epoch 10/40
750/750 [==============================] - 18s 24ms/step - loss: 0.2350 - val_los

seq2seq 기계 번역기 동작시키기  
- 전체적인 번역 동작 단계 정리
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음
2. 상태와 \<SOS\>에 해당하는 \t를 디코더로 보냄
3. 디코더가 \<EOS\>에 해당하는 \n이 나올 때까지 다음 문자를 예측하는 행동을 반복 

In [19]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [20]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(
    inputs=[decoder_inputs] + decoder_states_inputs,
    outputs=[decoder_outputs] + decoder_states)

In [21]:
# 인덱스로부터 단어를 얻을 수 있음
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [22]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [23]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    # '\t'와 '\n'을 빼고 출력
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) 
    # '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) 

1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Déguerpissez. 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: Compris ! 
번역 문장: Dis-le ! 
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: Goodbye.
정답 문장: Ciao. 
번역 문장: Cassez-vous ! 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: He is old.
정답 문장: Il est vieux. 
번역 문장: Il est fou. 
